# 总体分析

- 

In [1]:
import sys,os
import mlstock
# 修改当前工作目录，为了方便导入项目中的包
dir = '/Users/piginzoo/workspaces/quant_research/cta/'
sys.path.append(dir)
os.chdir(dir)
print("当前工作目录：",os.getcwd())

from utils import utils
utils.init_logger()
conf = utils.load_config()


当前工作目录： /Users/piginzoo/workspaces/quant_research/cta


Loading BokehJS ...

2023-03-17 10:49:38,665 - INFO - utils.py:34 P66984: 读取配置文件:./conf/config.yml


开始初始化日志：file=False, simple=False
日志：创建控制台处理器


# 分析市盈率(EP)

https://tushare.pro/document/2?doc_id=95

In [2]:
start_date = '20100101'
end_date = '20230401'
stocks = ['300896.SZ','300122.SZ','603127.SH','600085.SH','300777.SZ','600276.SH','603233.SH','600036.SH','600196.SH']
# stocks = ['300357.SZ']


In [3]:
from mlstock.data import data_loader
from mlstock.data.datasource import DataSource
from mlstock.data.stock_info import StocksInfo
from mlstock.factors.daily_indicator import DailyIndicator

# stocks_info = StocksInfo(stocks, start_date, end_date)
# stock_data = data_loader.load(datasource, stocks, start_date, end_date)

# df_daily_indicator = DailyIndicator(datasource, stocks_info).calculate(stock_data)

# from mlstock.factors.finance_indicator import FinanceIndicator
# df_finance = FinanceIndicator.test(stocks, start_date, end_date)


datasource = DataSource()
df_finance = datasource.fina_indicator(stocks,start_date, end_date)

stocks_info = StocksInfo(stocks, start_date, end_date)
stock_data = data_loader.load(datasource, stocks, start_date, end_date)
df_daily_basic = stock_data.df_daily_basic


2023-03-17 10:49:44,426 - INFO - utils.py:31 P66984: 读取配置文件:'conf/config.yml'


OperationalError: (pymysql.err.OperationalError) (2003, "Can't connect to MySQL server on '192.168.2.100' ([Errno 64] Host is down)")
(Background on this error at: http://sqlalche.me/e/14/e3q8)

In [ ]:
df_finance.columns

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
import talib
from mlstock.utils import df_utils
# stocks = ['600036.SH']
for stock in stocks:
    df1 = df_daily_basic[df_daily_basic.ts_code==stock]
    if len(df1)==0:
        print(f"{stock}数据为空")
        continue
    dates = df_utils.to_datetime(df1['trade_date'],'%Y%m%d')

    df_upper, df_middle, df_lower = talib.BBANDS(
                     df1.pe,
                     timeperiod=850,
                     nbdevup=1,
                     nbdevdn=1,
                     matype=0)

    fig = plt.figure(figsize=(20, 5), dpi=(100))
    plt.plot(dates,df1.pe)
    axis = fig.get_axes()[0]
    axis.set_title(stock+"/市盈率PE")
    plt.plot(dates,df_middle)
    plt.fill_between(dates, df_upper, df_lower, alpha=0.2)
    

    df1 = df_finance[df_finance.ts_code==stock]
    dates = df_utils.to_datetime(df1['ann_date'],'%Y%m%d')
    fig = plt.figure(figsize=(20, 5), dpi=(100))
    plt.plot(dates,df1.roe)
    axis = fig.get_axes()[0]
    axis.set_title(stock+"/净资产回报率ROE")

    df1 = df_finance[df_finance.ts_code==stock]
    fig = plt.figure(figsize=(20, 5), dpi=(100))
    plt.plot(dates,df1.grossprofit_margin)
    axis = fig.get_axes()[0]
    axis.set_title(stock+"/销售毛利率grossprofit_margin")
